In [1]:
%cd /kaggle/input/rtdetrzip/detrex-main

/kaggle/input/rtdetrzip/detrex-main


In [2]:
cp -r /kaggle/input/rtdetrzip/detrex-main /kaggle/working/

In [3]:
%cd /kaggle/working/detrex-main

/kaggle/working/detrex-main


In [4]:
!pip install -e .

Obtaining file:///kaggle/working/detrex-main
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.3/266.3 kB 9.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.2/94.2 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.9/74.9 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.2/63.2 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━

In [5]:
import os, json, cv2
from pathlib import Path

def create_data_pairs(input_path, dir_type):
    img_paths = Path(f"{input_path}/{dir_type}/images").glob("*.png")
    return [[str(img), str(img).replace("images", "labels").replace(".png", ".txt")] for img in img_paths if Path(str(img).replace("images", "labels").replace(".png", ".txt")).exists()]

def create_coco_format(data_pairs):
    data_list = []
    for i, (img_path, label_path) in enumerate(data_pairs):
        h, w = cv2.imread(img_path).shape[:2]
        item = {
            "file_name": os.path.basename(img_path),
            "image_id": i,
            "height": h,
            "width": w,
            "annotations": []
        }
        with open(label_path) as f:
            for line in f:
                cls, x_c, y_c, bw, bh = map(float, line.strip().split())
                x1 = (x_c - bw/2) * w
                y1 = (y_c - bh/2) * h
                x2 = (x_c + bw/2) * w
                y2 = (y_c + bh/2) * h
                item["annotations"].append({
                    "bbox": [x1, y1, x2, y2],
                    "category_id": int(cls),
                    "iscrowd": 0
                })
        data_list.append(item)
    return data_list

def convert_to_coco_json(data_list, output_path, class_names):
    coco = {
        "images": [],
        "annotations": [],
        "categories": [{"id": i, "name": name, "supercategory": "none"} for i, name in enumerate(class_names)]
    }
    ann_id = 1
    for img in data_list:
        coco["images"].append({
            "file_name": img["file_name"],
            "id": img["image_id"],
            "width": img["width"],
            "height": img["height"]
        })
        for ann in img["annotations"]:
            x1, y1, x2, y2 = ann["bbox"]
            w, h = x2 - x1, y2 - y1
            coco["annotations"].append({
                "id": ann_id,
                "image_id": img["image_id"],
                "category_id": ann["category_id"],
                "bbox": [x1, y1, w, h],
                "area": w * h,
                "iscrowd": ann["iscrowd"]
            })
            ann_id += 1
    with open(output_path, 'w') as f:
        json.dump(coco, f)

# Dataset prep
classes = ['ultra-thin', 'thin', 'poles', 'fence', 'wires', 'mesh', 'trees','forest','bungalow','car','miscellaneous']
data_path = "/kaggle/input/harshdata/dataset"
train_data = create_data_pairs(data_path, "train")
val_data = create_data_pairs(data_path, "val")
train_list = create_coco_format(train_data)
val_list = create_coco_format(val_data)

os.makedirs("/kaggle/working/my_dataset/annotations", exist_ok=True)
convert_to_coco_json(train_list, "/kaggle/working/my_dataset/annotations/instances_train.json", classes)
convert_to_coco_json(val_list, "/kaggle/working/my_dataset/annotations/instances_val.json", classes)


In [6]:
!mkdir -p /kaggle/working/my_dataset/train /kaggle/working/my_dataset/val
!cp /kaggle/input/harshdata/dataset/train/images/* /kaggle/working/my_dataset/train
!cp /kaggle/input/harshdata/dataset/val/images/* /kaggle/working/my_dataset/val

In [7]:
!pip install git+https://github.com/facebookresearch/detectron2.git

  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-9cys5fg_
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-9cys5fg_
  Resolved https://github.com/facebookresearch/detectron2.git to commit 536dc9d527074e3b15df5f6677ffe1f4e104a4ab
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for detectron2: filename=detectron2-0.6-cp311-cp311-linux_x86_64.whl size=6380996 sha256=d242f068a1ed11d338a6a64a17cb7de54c7089db6ffee28100f090d73a4cba82
  Stored in directory: /tmp/pip-ephem-wheel-cache-2abvkhzw/wheels/17/d9/40/60db98e485aa9455d653e29d1046601ce96fe23647f60c1c5a
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61396 sha256=bb488d3b216cdbd9e5d29fa0ab626252e3dad01c1b1fbf139a0464da91bc5853
  Stored in directory: /root/.cache/pip/

In [8]:
!pip install supervision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.5/181.5 kB 4.4 MB/s eta 0:00:00


In [9]:
!pip install datasets torchvision transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.12.0 which is incompatible.
bigframes 1.36.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.


In [10]:
!pip install -U "transformers>=4.37.0" torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 77.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 106.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.2/865.2 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 96.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 65.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 48.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.2/200.2 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.2/158.2 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.6

In [11]:
from torch.utils.data import Dataset
from pycocotools.coco import COCO
from PIL import Image
import os
import torch

In [12]:
class CocoDetectionHFDataset(Dataset):
    def __init__(self, image_dir, ann_file, processor):
        self.image_dir = image_dir
        self.coco = COCO(ann_file)
        self.ids = list(self.coco.imgs.keys())
        self.processor = processor

    def __len__(self):
        return len(self.ids)

    def __getitem__(self, idx):
        image_id = self.ids[idx]
        ann_ids = self.coco.getAnnIds(imgIds=image_id)
        anns = self.coco.loadAnns(ann_ids)
        file_name = self.coco.imgs[image_id]['file_name']
        image_path = os.path.join(self.image_dir, file_name)
        image = Image.open(image_path).convert("RGB")

        boxes = [ann["bbox"] for ann in anns]
        class_labels = [ann["category_id"] for ann in anns]

        labels = {
            "class_labels": torch.tensor(class_labels, dtype=torch.int64),
            "boxes": torch.tensor(boxes, dtype=torch.float32)
        }

        encoding = self.processor(images=image, return_tensors="pt")

        return {
            "pixel_values": encoding["pixel_values"].squeeze(0),
            "labels": labels,
            "image_id": image_id,
            "image": image
        }


In [13]:
from transformers import AutoModelForObjectDetection, AutoImageProcessor

checkpoint = "PekingU/rtdetr_r50vd_coco_o365"

# Your class names
id2label = {
    0: 'ultra-thin', 1: 'thin', 2: 'poles', 3: 'fence', 4: 'wires', 5: 'mesh',
    6: 'trees', 7: 'forest', 8: 'bungalow', 9: 'car', 10: 'miscellaneous'
}
label2id = {v: k for k, v in id2label.items()}

processor = AutoImageProcessor.from_pretrained(checkpoint)
model = AutoModelForObjectDetection.from_pretrained(
    checkpoint,
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True
).to("cuda" if torch.cuda.is_available() else "cpu")


2025-05-03 01:08:37.799243: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746234518.045087      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746234518.114504      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


preprocessor_config.json:   0%|          | 0.00/841 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


config.json:   0%|          | 0.00/5.11k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/172M [00:00<?, ?B/s]

Some weights of RTDetrForObjectDetection were not initialized from the model checkpoint at PekingU/rtdetr_r50vd_coco_o365 and are newly initialized because the shapes did not match:
- model.decoder.class_embed.0.bias: found shape torch.Size([80]) in the checkpoint and torch.Size([11]) in the model instantiated
- model.decoder.class_embed.0.weight: found shape torch.Size([80, 256]) in the checkpoint and torch.Size([11, 256]) in the model instantiated
- model.decoder.class_embed.1.bias: found shape torch.Size([80]) in the checkpoint and torch.Size([11]) in the model instantiated
- model.decoder.class_embed.1.weight: found shape torch.Size([80, 256]) in the checkpoint and torch.Size([11, 256]) in the model instantiated
- model.decoder.class_embed.2.bias: found shape torch.Size([80]) in the checkpoint and torch.Size([11]) in the model instantiated
- model.decoder.class_embed.2.weight: found shape torch.Size([80, 256]) in the checkpoint and torch.Size([11, 256]) in the model instantiated
- 

In [14]:
train_dataset = CocoDetectionHFDataset(
    image_dir="/kaggle/working/my_dataset/train",
    ann_file="/kaggle/working/my_dataset/annotations/instances_train.json",
    processor=processor
)

val_dataset = CocoDetectionHFDataset(
    image_dir="/kaggle/working/my_dataset/val",
    ann_file="/kaggle/working/my_dataset/annotations/instances_val.json",
    processor=processor
)

loading annotations into memory...
Done (t=0.54s)
creating index...
index created!
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [15]:
def collate_fn(batch):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    pixel_values = torch.stack([x["pixel_values"] for x in batch]).to(device)
    labels = [
        {
            "boxes": x["labels"]["boxes"].to(device),
            "class_labels": x["labels"]["class_labels"].to(device)
        }
        for x in batch
    ]
    image_ids = [x["image_id"] for x in batch]
    images = [x["image"] for x in batch]

    return {
        "pixel_values": pixel_values,
        "labels": labels,
        "image_ids": image_ids,
        "images": images
    }


In [16]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./rtdetr-finetuned",
    num_train_epochs=10,
    per_device_train_batch_size=4,
    learning_rate=5e-5,
    warmup_steps=300,
    max_grad_norm=0.1,
    dataloader_num_workers=2,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    remove_unused_columns=False,
    logging_dir="./logs",
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    load_best_model_at_end=True
)


In [17]:
from torch.utils.data import DataLoader

sample_batch = next(iter(DataLoader(train_dataset, batch_size=2, collate_fn=collate_fn)))
for key in sample_batch:
    print(f"{key}: {type(sample_batch[key])}")

# Check labels content
print("Labels batch size:", len(sample_batch["labels"]))
print("First label entry:", sample_batch["labels"][0])


pixel_values: <class 'torch.Tensor'>
labels: <class 'list'>
image_ids: <class 'list'>
images: <class 'list'>
Labels batch size: 2
First label entry: {'boxes': tensor([[ 624.1125,  572.2610,  306.1842,  145.8572],
        [ 865.2558,  594.9420,   21.9613,   46.3099],
        [ 624.7097,  574.1477,  303.5590,  143.5618],
        [ 624.9739,  574.3525,  305.6527,  144.1904],
        [   1.8541,  372.1221,  660.8978,  346.1913],
        [ 865.8251,  483.2651,  224.8300,  224.0770],
        [ 671.9546,  563.8519,  257.5620,  154.5598],
        [ 742.8917,  425.9268,  112.7495,  157.3973],
        [1084.2782,  462.2608,  114.4871,  101.6399],
        [1144.0568,  334.8235,  134.8342,  216.8926],
        [   2.3384,  339.1874, 1276.0464,  378.6399],
        [ 741.8254,  425.9684,  136.2911,  216.4180],
        [   2.3799,  337.7093, 1275.2808,  380.2589],
        [   2.0905,  371.8627,  659.3215,  346.4623],
        [ 622.7491,  574.2745,  306.4426,  144.5985],
        [ 622.3068,  576.0103, 

In [18]:
from torch.utils.data import DataLoader

loader = DataLoader(train_dataset, batch_size=2, collate_fn=collate_fn)
sample_batch = next(iter(loader))

model.train()
output = model(**sample_batch)
print("✅ Loss:", output.loss.item())

✅ Loss: 90303.125


In [19]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True, collate_fn=collate_fn)

In [20]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5, weight_decay=1e-4)


In [21]:
from tqdm import tqdm

model.train()

for epoch in range(100):  
    print(f"🔁 Epoch {epoch+1}")
    running_loss = 0.0

    for batch in tqdm(train_loader):
        # Move to device (already handled in collate_fn)
        outputs = model(**batch)
        loss = outputs.loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1} Loss: {running_loss:.2f}")


🔁 Epoch 1


100%|██████████| 800/800 [06:08<00:00,  2.17it/s]


Epoch 1 Loss: 69072076.77
🔁 Epoch 2


100%|██████████| 800/800 [06:13<00:00,  2.14it/s]


Epoch 2 Loss: 69139036.57
🔁 Epoch 3


...

Epoch 98 Loss: 69162826.50
🔁 Epoch 99


100%|██████████| 800/800 [06:08<00:00,  2.17it/s]


Epoch 99 Loss: 69246163.84
🔁 Epoch 100


100%|██████████| 800/800 [06:08<00:00,  2.17it/s]

Epoch 100 Loss: 69121645.02


In [22]:
val_loader = DataLoader(
    val_dataset,
    batch_size=2,
    shuffle=False,
    collate_fn=collate_fn
)

In [23]:
model.eval()

RTDetrForObjectDetection(
  (model): RTDetrModel(
    (backbone): RTDetrConvEncoder(
      (model): RTDetrResNetBackbone(
        (embedder): RTDetrResNetEmbeddings(
          (embedder): Sequential(
            (0): RTDetrResNetConvLayer(
              (convolution): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
              (normalization): RTDetrFrozenBatchNorm2d()
              (activation): ReLU()
            )
            (1): RTDetrResNetConvLayer(
              (convolution): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
              (normalization): RTDetrFrozenBatchNorm2d()
              (activation): ReLU()
            )
            (2): RTDetrResNetConvLayer(
              (convolution): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
              (normalization): RTDetrFrozenBatchNorm2d()
              (activation): ReLU()
            )
          )
          (pooler): MaxPool2d(

In [24]:
from PIL import Image

In [25]:
import matplotlib.pyplot as plt
import torchvision.transforms.functional as F

In [26]:
!pip install -q pycocotools

In [27]:
predictions = []

for batch in tqdm(val_loader):
    with torch.no_grad():
        outputs = model(pixel_values=batch["pixel_values"], labels=batch["labels"])

    target_sizes = torch.tensor([img.size[::-1] for img in batch["images"]])  # (H, W)
    results = processor.post_process_object_detection(outputs, target_sizes=target_sizes, threshold=0.5)

    for i, pred in enumerate(results):
        image_id = batch["image_ids"][i]
        for box, score, label in zip(pred["boxes"], pred["scores"], pred["labels"]):
            box = box.tolist()
            x, y, x2, y2 = box
            w, h = x2 - x, y2 - y
            predictions.append({
                "image_id": int(image_id),
                "category_id": int(label.item()),
                "bbox": [x, y, w, h],
                "score": float(score)
            })


100%|██████████| 50/50 [00:09<00:00,  5.18it/s]


In [28]:
print(f"Total predictions: {len(predictions)}")

Total predictions: 0


In [29]:
with open("predictions.json", "w") as f:
    json.dump(predictions, f)

coco_gt = COCO("/kaggle/working/my_dataset/annotations/instances_val.json")
coco_dt = coco_gt.loadRes("predictions.json")

coco_eval = COCOeval(coco_gt, coco_dt, iouType="bbox")
coco_eval.evaluate()
coco_eval.accumulate()
coco_eval.summarize()

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
Loading and preparing results...


IndexError: list index out of range

In [ ]:
with open("predictions.json", "w") as f:
    json.dump(predictions, f)

In [ ]:
val_images = os.listdir("/kaggle/working/my_dataset/val")[:5]

for image_name in val_images:
    image_path = f"/kaggle/working/my_dataset/val/{image_name}"
    image = Image.open(image_path).convert("RGB")
    inputs = processor(images=image, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model(**inputs)

    target_sizes = torch.tensor([image.size[::-1]])  # (height, width)
    results = processor.post_process_object_detection(outputs, target_sizes=target_sizes, threshold=0.5)[0]

    draw = image.copy()
    draw = F.to_tensor(draw).permute(1, 2, 0).numpy()

    for box, label, score in zip(results["boxes"], results["labels"], results["scores"]):
        box = box.cpu().numpy()
        label_name = id2label[label.item()]
        x, y, w, h = box[0], box[1], box[2] - box[0], box[3] - box[1]
        plt.gca().add_patch(plt.Rectangle((x, y), w, h, edgecolor='lime', facecolor='none', linewidth=2))
        plt.text(x, y - 10, f"{label_name} ({score:.2f})", color='lime', fontsize=9)

    plt.imshow(draw)
    plt.axis("off")
    plt.show()
